# Bars Multiplicity determination with Scikit-learn classifiers

Here we test out scikit-learn classification models for multiplicity reconstruction.

In [ ]:
label = "nPN"
mode = "bars"
nmax = 4

In [2]:
import copy
import math
import multiprocessing
import sys
import time

import numpy as np
import pandas as pd
import sklearn
from joblib import Parallel, delayed
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting

sys.path.append("..")
from helpers import filename_for, with_timeout

Welcome to JupyROOT 6.18/04


Load ALL the classification models from scikit-learn.
Note that some models are very slow to train with large datasets or crash outright, so we give them a reduced number of (shuffled) rows to learn.
Note that `n_jobs=1` is used, as parallelism is introduced later.

In [3]:
models_a1 = [
    ("BaggingClassifier", sklearn.ensemble.BaggingClassifier(n_jobs=1), 25000),
    ("BernoulliNB", sklearn.naive_bayes.BernoulliNB(), 600000),
    ("CalibratedClassifierCV", sklearn.calibration.CalibratedClassifierCV(cv=5), 10000),  # Slow with some scalers
    ("ComplementNB", sklearn.naive_bayes.ComplementNB(), 600000),
    ("GaussianNB", sklearn.naive_bayes.GaussianNB(), 600000),
]

models_a2 = [
    ("LinearDiscriminantAnalysis", sklearn.discriminant_analysis.LinearDiscriminantAnalysis(), 150000),
    ("LinearSVC", sklearn.svm.LinearSVC(max_iter=20000), 60000),  # slow with unscaled data
    (
        "LogisticRegression",
        sklearn.linear_model.LogisticRegression(solver="lbfgs", multi_class="auto", max_iter=20000),
        5000,
    ),  # slow with unscaled data
    (
        "LogisticRegressionCV",
        sklearn.linear_model.LogisticRegressionCV(cv=5, solver="lbfgs", multi_class="auto", max_iter=20000),
        5000,
    ),
    ("MLPClassifier", sklearn.neural_network.MLPClassifier(), 25000),
    ("MultinomialNB", sklearn.naive_bayes.MultinomialNB(), 600000),
]

models_b = [
    ("NearestCentroid", sklearn.neighbors.NearestCentroid(), 600000),
    (
        "PassiveAggressiveClassifier",
        sklearn.linear_model.PassiveAggressiveClassifier(max_iter=1000, tol=1e-3, n_jobs=1),
        600000,
    ),
    ("Perceptron", sklearn.linear_model.Perceptron(n_jobs=1), 600000),
    ("QuadraticDiscriminantAnalysis", sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(), 50000),
    ("RidgeClassifier", sklearn.linear_model.RidgeClassifier(), 600000),
    ("RidgeClassifierCV", sklearn.linear_model.RidgeClassifierCV(), 200000),
    ("SGDClassifier", sklearn.linear_model.SGDClassifier(max_iter=5000, tol=1e-3, n_jobs=1), 25000),
]

# Run these sequential due to "buffer source array is read-only" with LokyBackend
models_s = [
    ("AdaBoostClassifier", sklearn.ensemble.AdaBoostClassifier(), 100000),
    ("DecisionTreeClassifier", sklearn.tree.DecisionTreeClassifier(), 600000),
    ("ExtraTreeClassifier", sklearn.tree.ExtraTreeClassifier(), 600000),
    ("ExtraTreesClassifier", sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, n_jobs=-1), 600000),
    ("RandomForestClassifier", sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1), 600000),
]

# Models that fail alot
models_o = [
    # ('NuSVC', sklearn.svm.NuSVC(), 'fast'),  # nu infeasible
    # ("RadiusNeighborsClassifier", sklearn.neighbors.RadiusNeighborsClassifier(radius=10, n_jobs=-1), MEDI),
    ("GaussianProcessClassifier", sklearn.gaussian_process.GaussianProcessClassifier(), 800),
    # ('GradientBoostingClassifier', sklearn.ensemble.GradientBoostingClassifier(), 'slow'),  # crashes
    # ('HistGradientBoostingClassifier', sklearn.ensemble.HistGradientBoostingClassifier(), 'slow'),  # crashes?
    ("KNeighborsClassifier", sklearn.neighbors.KNeighborsClassifier(n_jobs=10), 600000),
    (
        "LabelPropagation",
        sklearn.semi_supervised.LabelPropagation(),
        600000,
    ),  # requires too much memory to train with larger datasets
    ("LabelSpreading", sklearn.semi_supervised.LabelSpreading(), 600000),  # bit slow
    ("SVC", sklearn.svm.SVC(gamma="scale"), 20000),  # slow, not timeouted
]

Some models only work with properly scaled data, so we prepare ALL available scalers.

In [4]:
class UnscaledScaler(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X, copy=None):
        if isinstance(X, pd.DataFrame):
            return X.to_numpy()
        else:
            return X

    def fit_transform(self, X, y=None):
        return self.transform(X, y)


scalers = [
    ("Unscaled data", UnscaledScaler()),
    ("standard scaling", sklearn.preprocessing.StandardScaler()),
    ("min-max scaling", sklearn.preprocessing.MinMaxScaler()),
    ("max-abs scaling", sklearn.preprocessing.MaxAbsScaler()),
    ("robust scaling", sklearn.preprocessing.RobustScaler(quantile_range=(25, 75))),
    # ("power transformation (Yeo-Johnson)", sklearn.preprocessing.PowerTransformer(method="yeo-johnson")), # complains about shapes
    # ('power transformation (Box-Cox)', sklearn.preprocessing.PowerTransformer(method='box-cox')), # 'strictly zero' meh.
    ("quantile transformation (gaussian pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="normal")),
    ("quantile transformation (uniform pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="uniform")),
    ("sample-wise L2 normalizing", sklearn.preprocessing.Normalizer()),
]

## Data preprocessing

Load

In [5]:
files = [
    filename_for(15, 30, 600, 500, n, "inclxx", s, "bars.parquet") for n in range(1, nmax + 1) for s in range(6)
]  # 20)]
dfs = [pd.read_parquet(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1)
data.loc[data["nHits"] == 0, ["nPN", "nPP", "nPH"]] = 0
display(data)

,nPN,nPP,nPH,nHits,nClus,Edep,0,1,2,3,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
157010,3,3,3,48,22,846,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47244,1,1,1,16,4,356,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95309,2,2,2,41,13,611,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197501,4,4,4,65,28,1208,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,3,3,3,54,23,869,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28299,1,1,1,22,3,387,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167966,3,3,3,40,23,692,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182916,4,4,2,31,5,636,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162636,3,3,3,56,30,892,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Train-Test-Split

In [6]:
msk = np.random.rand(data.shape[0]) < 0.8
traindata = data[msk]
testdata = data[~msk]

print(traindata.shape)
print(testdata.shape)

(192221, 6006)
(47779, 6006)


Scale Features: nHits, nClus, and Edep normally, but HitE / HitT from all bars together

In [7]:
cols_tri = ["nHits", "nClus", "Edep"]
cols_e = [str(i) for i in range(0, 30 * 100 * 2, 2)]
cols_t = [str(i + 1) for i in range(0, 30 * 100 * 2, 2)]

scalers_trained = [
    (
        sname,
        copy.copy(scaler).fit(traindata[cols_tri]),
        copy.copy(scaler).fit(traindata[cols_e].values.reshape(-1, 1)),
        copy.copy(scaler).fit(traindata[cols_t].values.reshape(-1, 1)),
    )
    for sname, scaler in scalers
]

In [8]:
if mode == "bars":
    data_scaled = [
        (
            sname,
            np.concatenate(
                (
                    # s_tri.transform(traindata[cols_tri]),
                    s_e.transform(traindata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(traindata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
            np.concatenate(
                (
                    # s_tri.transform(testdata[cols_tri]),
                    s_e.transform(testdata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(testdata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
        )
        for sname, s_tri, s_e, s_t in scalers_trained
    ]
elif mode == "barstri":
    data_scaled = [
        (
            sname,
            np.concatenate(
                (
                    s_tri.transform(traindata[cols_tri]),
                    s_e.transform(traindata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(traindata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
            np.concatenate(
                (
                    s_tri.transform(testdata[cols_tri]),
                    s_e.transform(testdata[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
                    s_t.transform(testdata[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
                ),
                axis=1,
            ),
        )
        for sname, s_tri, s_e, s_t in scalers_trained
    ]

## Run all model/scaler combinations, in parallel.
Note that we use timeouts per task, as setting at timeout in joblib will throw everything.

In [9]:
@with_timeout(1200 * 3)
def train_model(mname, modelorg, speed, sname, x_train, x_test):
    # These get killed without error?
    if mname == "RadiusNeighborsClassifier" and sname != "Unscaled data":
        return (mname, sname, np.NaN, speed, np.NaN, "Skipped")
    try:
        model = sklearn.base.clone(modelorg)
        start = time.time()
        model.fit(x_train[0:speed], traindata.head(speed)[[label]].values.ravel())
        end = time.time()

        y_pred = model.predict(x_test)
        y_true = testdata[[label]].values.ravel()

        bac = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)
        return (mname, sname, bac, speed, (end - start), "ok")
    except Exception as err:
        return (mname, sname, np.NaN, speed, np.NaN, err)


def train_model_wrap(mname, modelorg, speed, sname, x_train, x_test):
    ret = train_model(mname, modelorg, speed, sname, x_train, x_test)
    if ret:
        return ret
    else:
        return (mname, sname, np.NaN, speed, np.NaN, "Timeout")

In [10]:
try:
    results_a1 = Parallel(n_jobs=10, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_a1
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  40 out of  40 | elapsed: 34.4min finished
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numb

In [11]:
try:
    results_a2 = Parallel(n_jobs=5, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_a2
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed: 125.2min finished
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
try:
    results_b = Parallel(n_jobs=5, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_b
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed: 40.0min
[Parallel(n_jobs=5)]: Done  56 out of  56 | elapsed: 76.8min finished
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:740: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:740: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:743: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
/home/jmayer/.pyen

In [13]:
try:
    results_s = Parallel(n_jobs=1, verbose=1)(
        delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
        for sname, x_train, x_test in data_scaled
        for mname, modelorg, speed in models_s
    )
except Exception as err:
    print(err)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 192.1min finished


In [16]:
results_o = []
for mname, modelorg, speed in models_o:
    try:
        tmp = Parallel(n_jobs=10, verbose=1)(
            delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
            for sname, x_train, x_test in data_scaled
        )
        results_o.extend(tmp)
    except Exception as err:
        print(err)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 17.4min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 66.3min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed:   49.7s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed:   47.0s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 out of   8 | elapsed: 60.7min finished
Exception ignored in: <function Pool.__del__ at 0x7f97c979f550>
Traceback (most recent call last):
  File "/home/jmayer/.pyenv/versions/3.8.5/lib/python3.8/multiprocessing/pool.py", line 268, in __del__
    self._change_notifier.put(None)
  File "/home/jmayer/

In [17]:
results = results_a1 + results_a2 + results_b + results_s + results_o
resultsdf = pd.DataFrame(results)
pd.options.display.max_rows = 999
resultsdf.columns = ["Model", "Scaler", "BAC", "Events", "Time", "Status"]

max_events = traindata.shape[0]
resultsdf.loc[resultsdf["Events"] > max_events, "Events"] = max_events
resultsdf["Speed"] = resultsdf["Events"] / resultsdf["Time"]
resultsdf["OptEvents"] = resultsdf["Speed"] * 1200
resultsdf["OptEvents"] = resultsdf["OptEvents"].apply(lambda x: (0 if math.isnan(x) else round(x, 0)))
resultsdf["OptEvents"] = resultsdf["OptEvents"].astype("int")
resultsdf.loc[resultsdf["OptEvents"] > max_events, "OptEvents"] = max_events

resultsdf.sort_values(by=["BAC", "Time"], ascending=[False, True], inplace=True)
# resultsdf.sort_values(by=["Model", "Scaler"], ascending=[True, True], inplace=True)
resultsdf.style.hide_index().format({"BAC": "{:.2%}", "Time": "{:.2f}"}).bar(subset=["BAC"], color="lightgreen").bar(
    subset=["Time"], color="lightblue"
)

Model,Scaler,BAC,Events,Time,Status,Speed,OptEvents
BernoulliNB,standard scaling,67.50%,192221,13.49,ok,14248.741559,192221
BernoulliNB,sample-wise L2 normalizing,67.08%,192221,12.77,ok,15047.886664,192221
BernoulliNB,min-max scaling,67.08%,192221,12.84,ok,14965.848557,192221
BernoulliNB,quantile transformation (uniform pdf),67.08%,192221,12.90,ok,14899.432509,192221
BernoulliNB,quantile transformation (gaussian pdf),67.08%,192221,13.04,ok,14743.963585,192221
BernoulliNB,robust scaling,67.08%,192221,13.08,ok,14690.537163,192221
BernoulliNB,Unscaled data,67.08%,192221,13.39,ok,14354.668391,192221
BernoulliNB,max-abs scaling,67.08%,192221,13.88,ok,13851.037494,192221
ExtraTreesClassifier,robust scaling,65.25%,192221,205.00,ok,937.660395,192221
ExtraTreesClassifier,standard scaling,65.06%,192221,205.76,ok,934.219035,192221
